In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

# 混合损失函数：MSE + MAE
def mixed_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))
    return mse + 0.5 * mae

# 数据生成（增强和归一化）
def generate_synthetic_data(num_samples=1000, image_size=(64, 64)):
    X = np.random.rand(num_samples, *image_size)
    y = np.random.rand(num_samples, *image_size)

    # 数据增强（旋转、平移、加噪声、对比度变化）
    X = np.array([np.rot90(x, np.random.randint(0, 4)) for x in X])
    y = np.array([np.rot90(y_, np.random.randint(0, 4)) for y_ in y])
    X += np.random.normal(0, 0.01, X.shape)
    # 对比度增强
    X = np.clip(X * np.random.uniform(0.8, 1.2), 0, 1)
    # 归一化
    X = (X - np.min(X)) / (np.max(X) - np.min(X))
    y = (y - np.min(y)) / (np.max(y) - np.min(y))
    return X, y

# 更深更宽的U-Net结构，解码器加残差连接
def build_unet(input_shape=(64, 64, 1), base_filters=64, depth=5):
    inputs = layers.Input(shape=input_shape)
    x = inputs
    skips = []
    # 编码器
    for i in range(depth):
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        skips.append(x)
        x = layers.MaxPooling2D((2, 2))(x)
    # Bottleneck
    x = layers.Conv2D(base_filters * 2**depth, (3, 3), activation='relu', padding='same')(x)
    x = layers.Conv2D(base_filters * 2**depth, (3, 3), activation='relu', padding='same')(x)
    # 解码器+残差连接
    for i in reversed(range(depth)):
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Concatenate()([x, skips[i]])
        res = layers.Conv2D(base_filters * 2**i, (1, 1), activation='linear', padding='same')(skips[i])
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
        x = layers.Add()([x, res])
        x = layers.Conv2D(base_filters * 2**i, (3, 3), activation='relu', padding='same')(x)
    outputs = layers.Conv2D(1, (1, 1), activation='linear')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss=mixed_loss, metrics=['mse', 'mae'])
    return model

# 生成数据并进行训练
X, y = generate_synthetic_data()

# 数据维度调整
X = X[..., np.newaxis]
y = y[..., np.newaxis]

result_dir = os.path.join('results')
os.makedirs(result_dir, exist_ok=True)

print("训练更深更宽U-Net模型...")
model = build_unet(input_shape=(64, 64, 1), base_filters=64, depth=5)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-5)
history = model.fit(X, y, epochs=120, batch_size=32, validation_split=0.2, callbacks=[early_stopping, lr_scheduler], verbose=1)

val_mse = history.history['val_mse'][-1]
val_mae = history.history['val_mae'][-1]
print(f"最终验证集MSE: {val_mse:.4f}, MAE: {val_mae:.4f}")

# 保存损失曲线
plt.figure()
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Val Mixed Loss')
plt.legend()
plt.title('更深更宽U-Net验证损失曲线')
plt.savefig(os.path.join(result_dir, "deeper_unet_val_loss.png"))
plt.close()

# 随机选取样本进行对比可视化
idx = np.random.randint(0, X.shape[0])
sample_input = X[idx:idx+1]
sample_true = y[idx]
sample_pred = model.predict(sample_input)[0]

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('Input Data')
plt.imshow(sample_input[0, :, :, 0], cmap='gray')
plt.subplot(1, 3, 2)
plt.title('True Property')
plt.imshow(sample_true[:, :, 0], cmap='gray')
plt.subplot(1, 3, 3)
plt.title('Predicted Property (Deeper U-Net)')
plt.imshow(sample_pred[:, :, 0], cmap='gray')
plt.savefig(os.path.join(result_dir, "comparison_deeper_unet.png"))
plt.close()

# 保存最终验证集MSE和MAE到txt
with open(os.path.join(result_dir, "metrics_deeper_unet.txt"), "w") as f:
    f.write(f"Deeper U-Net: MSE={val_mse:.4f}, MAE={val_mae:.4f}\n")